In [1]:
from pyspark.sql import SparkSession as Session
from pyspark import SparkConf as Conf
from pyspark import SparkContext as Context

In [2]:
import os
os.environ['SPARK_LOCAL_IP']='192.168.1.2'
os.environ['HADOOP_HOME']='/home/geno1664/Developments/Github_Samples/RDS-ENV/hadoop'
os.environ['LD_LIBRARY_PATH']='$LD_LIBRARY_PATH:$HADOOP_HOME/lib/native'
os.environ['PYSPARK_DRIVER_PYTHON']='jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']='notebook'
os.environ['PYSPARK_PYTHON']='python3'
os.environ['PYARROW_IGNORE_TIMEZONE']='1'

TypeError: str expected, not int

In [ ]:
configuration = Conf().setAppName('RDS_1').setMaster('spark://GenoMachine:7077')
configuration.set('spark.executor.memory','10G').set('spark.driver.memory', '2G').set('spark.cores.max', '8')

In [ ]:
context = Context(conf=configuration)

In [ ]:
session = Session(context)

In [ ]:
from Functions.IO import CSV_File
mainCSV = CSV_File(session, r'/home/geno1664/Developments/Github_Samples/RDS-ENV/Rural_Development_Study_No1/IO/County_Classificiations.csv')

In [ ]:
mainCSV.Schema()

In [ ]:
from databricks import koalas as ks
mainDF = mainCSV.GetSparkDF().to_koalas(index_col=['State', 'County'])